In [31]:
# ライブラリのインポート
import numpy as np
import pandas as pd
import psycopg2
import datetime

In [97]:
npyname = "kddi_12_new_average_marge"
datedir = "230301"
sttime = "2022-04-01 12:00:00"

In [98]:
#解析結果の読み込み
timeseries_sep = np.load("./" +  datedir+'/'+ npyname + "_01param.npy")

In [99]:
timeseries_sep.shape

(24, 18930)

In [100]:
timeseries_sep

array([[0.08185913, 0.0397979 , 0.20691438, ..., 0.10310875, 0.12635259,
        0.        ],
       [0.07853346, 0.03520688, 0.1911634 , ..., 0.10553632, 0.12657127,
        0.        ],
       [0.07863927, 0.03526049, 0.19116366, ..., 0.10551447, 0.12648949,
        0.        ],
       ...,
       [0.07892344, 0.03541299, 0.19087353, ..., 0.10550342, 0.12641607,
        0.        ],
       [0.07892344, 0.03541299, 0.19087353, ..., 0.10550342, 0.12641607,
        0.        ],
       [0.07892344, 0.03541299, 0.19087353, ..., 0.10550342, 0.12641607,
        0.12641607]])

In [101]:
#メッシュ情報の読み込み
csv_file = r'./'+ datedir+'/'+ npyname+'_header.csv'
df = pd.read_csv(csv_file, dtype={'date':str}, index_col=0, parse_dates=True)
df_i = df.loc["mesh"] #gridcode #end_mesh_area

In [102]:
df_i.values.shape

(18930,)

In [103]:
connection = psycopg2.connectconnection = psycopg2.connect(host="localhost", database="analysisdb", user="postgres",password="setouchi1311")

In [104]:
sql = "CREATE TABLE " + npyname + "_result (times TIMESTAMP NULL DEFAULT NULL,mesh BIGINT NULL DEFAULT NULL,tbran DOUBLE PRECISION NULL DEFAULT NULL )"
cur =connection.cursor()
cur.execute(sql)
cur.close()
connection.commit()

In [105]:
cur =connection.cursor()
meshcnt = 0
for mesh in df_i.values:
    dte = datetime.datetime.strptime(sttime, '%Y-%m-%d %H:%M:%S')
    for i in range(timeseries_sep.shape[0]):
        #負数は対象としない
        if timeseries_sep[i][meshcnt] < 0:
            dte = dte + datetime.timedelta(minutes=15)
            continue
        sql = "INSERT INTO " + npyname + "_result (times, mesh,tbran) VALUES ('"+ str(dte) + "',"  + str(mesh) + "," + str(timeseries_sep[i][meshcnt]) + ");"
        cur.execute(sql)
        dte = dte + datetime.timedelta(minutes=15)
    meshcnt += 1
cur.close()
connection.commit()

In [106]:
connection.close()

In [ ]:
#ConvertMeshtoRoad